In [28]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [29]:
import kaggle

In [30]:
!kaggle datasets download -d nishchay331/retail-store


Dataset URL: https://www.kaggle.com/datasets/nishchay331/retail-store
License(s): other
retail-store.zip: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
import zipfile

with zipfile.ZipFile('retail-store.zip', 'r') as zip_ref:
    zip_ref.extractall('data_csv')  # Replace 'datasets' with your desired folder name



In [32]:
import pandas as pd
import numpy as np

# Load the CSVs into pandas DataFrames
df1 = pd.read_csv('data_csv/data_2021.csv')
df2 = pd.read_csv('data_csv/data_2022.csv')

# Check the shape and top rows
print(df1.shape, df2.shape)
df1.head()
df2.head()

(1661026, 12) (2534161, 12)


,user_id,bill_id,line_item_amount,bill_discount,transaction_date,description,inventory_category,colour,size,zone_name,store_name,year
0,519644808,741961800,559.6,0.0,2022-04-20,MBL ITA16BLT004 Regular Casual Tan 32,MENS BELT,Tan,32,North,North_6023,2022
1,519644808,741961800,559.6,0.0,2022-04-20,MBL ITA16BLT004 Regular Casual Tan 32,MENS BELT,Tan,32,North,North_6023,2022
2,504795522,692673877,699.5,0.0,2022-01-27,MBL ITA16BLT004 Regular Casual Tan 34,MENS BELT,Tan,34,East,East_6510,2022
3,12626591,768369011,519.6,0.0,2022-10-09,MBL ITA16BLT001 Regular Casual Dark Brown 32,MENS BELT,Dark Brown,32,South,South_6017,2022
4,12626591,768369011,519.6,0.0,2022-10-09,MBL ITA16BLT001 Regular Casual Dark Brown 32,MENS BELT,Dark Brown,32,South,South_6017,2022


In [33]:
df1.isnull().sum()

user_id                    0
bill_id                    0
line_item_amount           0
bill_discount              0
transaction_date           0
description               10
inventory_category    414103
colour                    22
size                      22
zone_name                  0
store_name                 0
year                       0
dtype: int64

In [34]:
df2.isnull().sum()

user_id                    0
bill_id                    0
line_item_amount           0
bill_discount              0
transaction_date           0
description                4
inventory_category    208257
colour                 15313
size                   15313
zone_name                  0
store_name                 0
year                       0
dtype: int64

In [35]:
print("DF1 duplicates:", df1.duplicated().sum())
print("DF2 duplicates:", df2.duplicated().sum())





DF1 duplicates: 348289
DF2 duplicates: 558608


In [36]:
# Dropping  duplicates 
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

In [37]:
# Drop the specified columns not needed for the analysis on data_2021
df1 = df1.drop(columns=['description', 'inventory_category', 'colour', 'size'])

# Verify the updated DataFrame
print(df1.head())

df1.isnull().sum()


     user_id    bill_id  line_item_amount  bill_discount transaction_date  \
0  432158864  605654505          779.4000            0.0       2021-02-12   
1   10388511  642524803         1402.3148            0.0       2021-08-22   
2  469000103  600607021         1154.7635            0.0       2021-01-22   
3  495324431  654044746           99.0000            0.0       2021-10-12   
4  477957785  636941492         1026.1622            0.0       2021-07-28   

  zone_name  store_name  year  
0      East   East_7310  2021  
1     South  South_7045  2021  
2     North  North_7064  2021  
3     North  North_7118  2021  
4     North  North_7013  2021  


user_id             0
bill_id             0
line_item_amount    0
bill_discount       0
transaction_date    0
zone_name           0
store_name          0
year                0
dtype: int64

In [38]:
# Drop the specified columns not needed for the analysis on data_2022
df2 = df2.drop(columns=['description', 'inventory_category', 'colour', 'size'])

# Verify the updated DataFrame
print(df2.head())



df1.isnull().sum()

      user_id    bill_id  line_item_amount  bill_discount transaction_date  \
0   519644808  741961800          559.6000            0.0       2022-04-20   
2   504795522  692673877          699.5000            0.0       2022-01-27   
3    12626591  768369011          519.6000            0.0       2022-10-09   
10  496507098  779140853          559.6000            0.0       2022-11-10   
12  518922122  740235900          937.1713            0.0       2022-07-04   

   zone_name  store_name  year  
0      North  North_6023  2022  
2       East   East_6510  2022  
3      South  South_6017  2022  
10     South  South_6028  2022  
12     South  South_6028  2022  


user_id             0
bill_id             0
line_item_amount    0
bill_discount       0
transaction_date    0
zone_name           0
store_name          0
year                0
dtype: int64

In [39]:
df1['transaction_date'] = pd.to_datetime(df1['transaction_date'])
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'])


In [40]:
# 4. Merge/Append Data
df_combined = pd.concat([df1, df2], ignore_index=True)

In [41]:


df_combined.to_csv('df_combined.csv', index=False)
print('done')

done


In [42]:
# 5. Create net_amount column
df_combined['net_amount'] = df_combined['line_item_amount'] - df_combined['bill_discount']

In [43]:

# 6. Key Metrics

In [44]:

## 6.1 CLV
customer_clv = df_combined.groupby('user_id')['net_amount'].sum().reset_index()
customer_clv.columns = ['user_id', 'CLV']

In [45]:
customer_clv.to_csv('customer_clv.csv', index=False)

In [46]:
## 6.2 Retention Rate (example year-based)
df_combined['year'] = df_combined['transaction_date'].dt.year
customers_2021 = set(df_combined[df_combined['year'] == 2021]['user_id'].unique())
customers_2022 = set(df_combined[df_combined['year'] == 2022]['user_id'].unique())
retained_customers = customers_2021.intersection(customers_2022)
retention_rate_2021_to_2022 = len(retained_customers) / len(customers_2021) 
print('Retention Rate:', retention_rate_2021_to_2022)

Retention Rate: 0.25137634146628995


In [47]:
## 6.3 Purchase Frequency
purchase_counts = df_combined.groupby('user_id')['bill_id'].nunique().reset_index()
purchase_counts.columns = ['user_id', 'purchase_count']
avg_purchase_freq = purchase_counts['purchase_count'].mean()
print('Average purchase frequency:', avg_purchase_freq)

Average purchase frequency: 1.7752459425151872


In [48]:

## 6.4 AOV
total_orders = df_combined['bill_id'].nunique()
total_revenue = df_combined['net_amount'].sum()
aov = total_revenue / total_orders
print('AOV:',aov)

AOV: 4381.1896880088125


In [49]:
import pandas as pd

# 2. Create a dictionary of key metrics
key_metrics = {
    'Retention Rate (2021-2022)': retention_rate_2021_to_2022,
    'Average Purchase Frequency': avg_purchase_freq,
    'AOV': aov,  
    'Total Revenue': df_combined['net_amount'].sum()  
}

# 3. Convert to DataFrame
key_metrics_df = pd.DataFrame([key_metrics])

# 4. Save to CSV
key_metrics_df.to_csv('Key_metrics.csv', index=False)

# 5. (Optional) Display the DataFrame
key_metrics_df


,Retention Rate (2021-2022),Average Purchase Frequency,AOV,Total Revenue
0,0.251376,1.775246,4381.189688,4.117446e+09


In [50]:
#RFM analysis

In [51]:
## 6.5 RFM
snapshot_date = df_combined['transaction_date'].max() + pd.Timedelta(days=1)
rfm = df_combined.groupby('user_id').agg({
    'transaction_date': 'max',
    'bill_id': 'nunique',
    'net_amount': 'sum'
}).reset_index()

rfm.columns = ['user_id', 'LastPurchaseDate', 'Frequency', 'Monetary']
rfm['Recency'] = (snapshot_date - rfm['LastPurchaseDate']).dt.days
rfm.drop('LastPurchaseDate', axis=1, inplace=True)

# R, F, M scoring
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# 7. Results
print("CLV sample:")
print(customer_clv.head())

print("Yearly retention rate (2021 -> 2022):", retention_rate_2021_to_2022 )
print("Average Purchase Frequency:", avg_purchase_freq)
print("Average Order Value:", aov)

print("RFM sample:")
print(rfm.head())

CLV sample:
   user_id         CLV
0    50049   8805.0001
1    50174   -502.0000
2    50511  16246.0021
3    50565   6915.0010
4    53762   5397.9999
Yearly retention rate (2021 -> 2022): 0.25137634146628995
Average Purchase Frequency: 1.7752459425151872
Average Order Value: 4381.1896880088125
RFM sample:
   user_id  Frequency    Monetary  Recency R_Score F_Score M_Score RFM_Score
0    50049          2   8805.0001      205       3       4       4       344
1    50174          1   -502.0000      386       2       1       1       211
2    50511          3  16246.0021      300       3       5       5       355
3    50565          3   6915.0010      138       4       5       4       454
4    53762          1   5397.9999      701       1       1       3       113


In [52]:
rfm.describe()

,user_id,Frequency,Monetary,Recency
count,5.293920e+05,529392.000000,5.293920e+05,529392.000000
mean,4.620010e+08,1.775246,7.777689e+03,291.333407
std,9.384413e+07,2.388799,1.506115e+04,203.070381
min,5.004900e+04,1.000000,-3.966734e+06,1.000000
25%,4.479641e+08,1.000000,2.399001e+03,115.000000
50%,4.943137e+08,1.000000,4.802000e+03,258.000000
75%,5.150882e+08,2.000000,8.860000e+03,435.000000
max,5.359105e+08,216.000000,1.414872e+06,730.000000


In [53]:
rfm.to_csv('rfm_analysis.csv', index=False)
print('done')


done
